<a href="https://colab.research.google.com/github/Veriny/MIT-6.S191/blob/master/tf2_0beta_moviereviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
!pip install -q tensorflow==2.0.0-beta1

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfdt

In [36]:
print(tf.test.is_gpu_available())

False


In [0]:
#Splitting a data-set
(train_data, validation_data), test_data = tfdt.load(name = 'imdb_reviews', split = (tfdt.Split.TRAIN.subsplit([6, 4]),tfdt.Split.TEST ), as_supervised=True)

In [0]:
#Check the data and its format

In [39]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(100)))
train_examples_batch

<tf.Tensor: id=1260, shape=(100,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story

In [0]:
#Use a pre-trained text embedding model
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype = tf.string, trainable = True)

In [0]:
model = tf.keras.Sequential()

In [47]:
print(tf.keras.__version__)
print(tf.__version__)


2.2.4-tf
2.0.0-beta1


In [48]:
model.add(hub_layer)
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 32)                672       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 400,725
Trainable params: 400,725
Non-trainable params: 0
_________________________________________________________________


In [0]:
#loss function, optimizer.
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [52]:
histroy = model.fit(train_data.shuffle(10000).batch(512), epochs = 40, validation_data = validation_data.batch(512), verbose = 1)

Epoch 1/40
30/30 [==============================] - 7s 236ms/step - loss: 0.7287 - accuracy: 0.5563 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/40
30/30 [==============================] - 7s 230ms/step - loss: 0.6510 - accuracy: 0.6242 - val_loss: 0.6230 - val_accuracy: 0.6536
Epoch 3/40
30/30 [==============================] - 7s 228ms/step - loss: 0.5967 - accuracy: 0.6837 - val_loss: 0.5802 - val_accuracy: 0.6997
Epoch 4/40
30/30 [==============================] - 7s 233ms/step - loss: 0.5555 - accuracy: 0.7245 - val_loss: 0.5465 - val_accuracy: 0.7282
Epoch 5/40
30/30 [==============================] - 7s 233ms/step - loss: 0.5193 - accuracy: 0.7557 - val_loss: 0.5161 - val_accuracy: 0.7540
Epoch 6/40
30/30 [==============================] - 7s 235ms/step - loss: 0.4845 - accuracy: 0.7833 - val_loss: 0.4874 - val_accuracy: 0.7726
Epoch 7/40
30/30 [==============================] - 7s 234ms/step - loss: 0.4503 - accuracy: 0.8048 - val_loss: 0.4598 - val_accuracy: 0.788

In [54]:
results = model.evaluate(test_data.batch(512), verbose = 0)
for name, label in zip(model.metrics_names, results):
  if name == 'loss':
    name = '| || || |_'
  print("{}: {}".format(name, label))

| || || |_: 0.4734886775211412
accuracy: 0.8498799800872803
